# 必要的基本方法

这一节介绍pandas数据结构内置的常用方法。很重要的一节。

创建本节要用到的数据结构。

In [2]:
import numpy as np
import pandas as pd

In [7]:
index  = pd.date_range('1/1/2000', periods=8)
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=['A', 'B', 'C'])
wp = pd.Panel(np.random.randn(2,5,4), items=['Item1', 'Item2'], major_axis=pd.date_range('1/1/2000',periods=5),
             minor_axis=['A', 'B', 'C', 'D'])

## Head() Tail()

想要预览Series或DataFrame对象，可以使用head()和tail()方法。默认显示的行数是5，你也可以自己设置显示的行数。

In [8]:
long_series = pd.Series(np.random.randn(1000))

In [9]:
long_series.head()

0   -0.229733
1    0.355929
2   -0.016854
3    0.790954
4   -1.099537
dtype: float64

In [10]:
long_series.tail(3)

997   -1.717307
998   -0.104273
999    0.377712
dtype: float64

## 属性和 numpy数组

pandas对象有很多属性，你可以通过这些属性访问数据。

* **shape**: 显示对象的维度，同ndarray
* 坐标label
* * **Series**: 索引
* * **DataFrame**:索引(行)和列
* * **Panel**:  items, major_axis and minor_axis

这些属性都可以安全滴进行赋值！

In [11]:
df[:2]

,A,B,C
2000-01-01,0.961850,0.832213,0.698852
2000-01-02,0.945025,-0.367759,0.061104


In [14]:
df.columns = [x.lower() for x in df.columns] #将列名重置为小写

In [15]:
df

,a,b,c
2000-01-01,0.961850,0.832213,0.698852
2000-01-02,0.945025,-0.367759,0.061104
2000-01-03,1.341735,-0.520314,0.621666
2000-01-04,-0.749680,0.737349,2.863382
2000-01-05,-0.395510,-1.115914,-0.739005
2000-01-06,0.868806,1.731622,-1.167939
2000-01-07,-2.260882,-0.748856,-1.483881
2000-01-08,0.002094,-0.744713,0.701304


只想得到对象中的数据而忽略列名索引这些乱七八糟的，使用values属性就可以

In [17]:
s.values

array([-1.0891997 ,  2.14039355, -0.42522808, -0.99005429,  0.44079549])

In [19]:
df.values

array([[  9.61849648e-01,   8.32213212e-01,   6.98851835e-01],
       [  9.45024544e-01,  -3.67758714e-01,   6.11041983e-02],
       [  1.34173499e+00,  -5.20313703e-01,   6.21666043e-01],
       [ -7.49680310e-01,   7.37349183e-01,   2.86338159e+00],
       [ -3.95509934e-01,  -1.11591412e+00,  -7.39005372e-01],
       [  8.68805636e-01,   1.73162237e+00,  -1.16793946e+00],
       [ -2.26088188e+00,  -7.48856000e-01,  -1.48388067e+00],
       [  2.09428537e-03,  -7.44713066e-01,   7.01303507e-01]])

In [20]:
type(df.values)

numpy.ndarray

In [21]:
wp.values

array([[[-1.70028729,  0.0228584 , -1.31088058, -1.81587031],
        [ 0.97998246, -0.91700322, -0.61644728,  0.10122091],
        [-0.0380311 ,  0.42598951,  0.43708298,  1.4847927 ],
        [ 0.11626489, -0.82049708, -0.3390273 , -0.64795144],
        [ 1.69714832,  3.5289194 , -1.04279508, -0.70495814]],

       [[-1.67817219,  0.12514369,  0.68422357,  0.08062269],
        [ 1.06218021, -1.0658129 ,  1.20032917, -0.02492852],
        [-1.46904051, -0.94881081, -0.62609046, -0.86555444],
        [-1.90217969,  0.3482584 ,  0.55615558,  0.76624635],
        [-0.46576376, -0.52609712,  0.46101983, -0.4530962 ]]])

如果DataFrame或Panel对象的数据类型相同(比如都是 int64)，修改object.values相当于直接修改原对象的值。如果数据类型不相同，则根本不能对values属性返回值进行赋值。

**注意**:

如果对象内数据类型不同，values返回的ndarray的dtype将是能够兼容所有数据类的类型。比如，数据是int和float，dtype将是float。

## 提高运算速度

pandas从0.11.0版本开始使用numexpr库对二值数值类型操作加速，用bottleneck库对布尔操作加速。

加速效果对大数据尤其明显。

这里有一个速度的简单对比，使用100,000行\* 100列的DataFrame:

![](https://ooo.0o0.ooo/2016/04/14/570f692c2e8a1.png)

所以，在安装pandas后也要顺便安装numexpr, bottleneck。

## 灵活的二元运算

在所有的pandas对象之间的二元运算中，大家最感兴趣的一般是下面两个：
* 广播(broadcasting)
* 缺失值计算

二者也可以结合使用，我们下面一一为大家介绍：

### 广播

DataFrame对象内置add(),sub(),mul(),div()以及radd(), rsub(),...等方法。

至于广播计算，Series的输入是最有意思的。

In [67]:
df = pd.DataFrame({'one' : pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
                       'two' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
                       'three' : pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})

In [68]:
df

,one,three,two
a,-0.819889,NaN,-0.748560
b,1.206005,-0.586788,0.630745
c,-0.620845,-0.080498,-0.227499
d,NaN,-1.495108,-1.590267


In [69]:
row = df.ix[1]
row

one      1.206005
three   -0.586788
two      0.630745
Name: b, dtype: float64

In [70]:
column = df['two']

In [71]:
column

a   -0.748560
b    0.630745
c   -0.227499
d   -1.590267
Name: two, dtype: float64

In [72]:
df.sub(row, axis='columns')

,one,three,two
a,-2.025895,NaN,-1.379305
b,0.000000,0.000000,0.000000
c,-1.826850,0.506291,-0.858244
d,NaN,-0.908320,-2.221013


In [73]:
df.sub(row, axis=1)

,one,three,two
a,-2.025895,NaN,-1.379305
b,0.000000,0.000000,0.000000
c,-1.826850,0.506291,-0.858244
d,NaN,-0.908320,-2.221013


In [74]:
df.sub(row, axis='index')

,one,three,two
a,NaN,NaN,NaN
b,NaN,NaN,NaN
c,NaN,NaN,NaN
d,NaN,NaN,NaN
one,NaN,NaN,NaN
three,NaN,NaN,NaN
two,NaN,NaN,NaN


In [75]:
df.sub(row, axis=0)

,one,three,two
a,NaN,NaN,NaN
b,NaN,NaN,NaN
c,NaN,NaN,NaN
d,NaN,NaN,NaN
one,NaN,NaN,NaN
three,NaN,NaN,NaN
two,NaN,NaN,NaN


### 填充缺失值

在Series和DataFrame中，算术运算方法(比如add())有一个fill_value参数，含义很明显，计算前用一个值来代替缺失值，然后再参与运算。注意，如果参与运算的两个object同一位置(同行同列)都是NaN，fill_value不起作用，计算结果还是NaN。

看例子：

In [76]:
df

,one,three,two
a,-0.819889,NaN,-0.748560
b,1.206005,-0.586788,0.630745
c,-0.620845,-0.080498,-0.227499
d,NaN,-1.495108,-1.590267


In [91]:
df2 = pd.DataFrame({'one' : pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
                       'two' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
                       'three' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd'])})

In [92]:
df2

,one,three,two
a,-0.745516,-0.657512,-1.358255
b,0.249448,-0.829815,0.258827
c,-0.117322,-0.016284,0.144683
d,NaN,0.213930,1.766173


In [93]:
df + df2

,one,three,two
a,-1.565406,NaN,-0.358255
b,1.455453,-1.416603,0.889572
c,-0.738167,-0.096782,-0.082815
d,NaN,-1.281178,0.175906


In [95]:
 df.add(df2, fill_value=0) #注意['a', 'three']不是NaN

,one,three,two
a,-1.565406,-0.657512,-0.358255
b,1.455453,-1.416603,0.889572
c,-0.738167,-0.096782,-0.082815
d,NaN,-1.281178,0.175906


### 灵活的比较操作

pandas引入了二元比较运算方法：eq, ne, lt, gt, le。

In [96]:
df.gt(df2)

,one,three,two
a,False,False,True
b,True,True,True
c,False,False,False
d,False,False,False


In [97]:
df2.ne(df)

,one,three,two
a,True,True,True
b,True,True,True
c,True,True,True
d,True,True,True


意思操作返回一个和输入对象同类型的对象，值类型为bool，返回结果可以用于检索。

### 布尔降维 Boolean Reductions

pandas提供了三个方法(any(), all(), bool())和一个empty属性来对布尔结果进行降维。

In [100]:
df>0

,one,three,two
a,False,False,True
b,True,False,True
c,False,False,False
d,False,False,False


In [101]:
(df>0).all() #与操作

one      False
three    False
two      False
dtype: bool

In [103]:
(df > 0).any()#或操作

one       True
three    False
two       True
dtype: bool

同样可以对降维后的结果再进行降维。

In [104]:
(df > 0).any().any()

True

使用empty属性检测一个pandas对象是否为空。

In [105]:
df.empty

False

In [106]:
pd.DataFrame(columns=list('ABC')).empty

True

对于只含有一个元素的pandas对象，对其进行布尔检测，使用bool():

In [108]:
pd.Series([True]).bool()

True

In [109]:
pd.Series([False]).bool()

False

In [111]:
pd.DataFrame([[True]]).bool()

True

In [112]:
pd.DataFrame([[False]]).bool()

False

### 比较对象是否相等

一个问题通常有多种解法。一个最简单的例子：df+df和df\*2。为了检测两个计算结果是否相等，你可能想到：(df+df == df\*2).all()，然而，这样计算得到的结果是False：

In [113]:
df + df == df*2

,one,three,two
a,True,False,True
b,True,True,True
c,True,True,True
d,False,True,True


In [114]:
(df+df == df*2).all()

one      False
three    False
two       True
dtype: bool

为什么df + df == df\*2 返回的结果含有False？因为NaN和NaN比较厚结果为False！

In [116]:
np.nan == np.nan

False

还好pandas提供了equals()方法解决上面NaN之间不想等的问题。

In [119]:
(df+df).equals(df*2)

True

**注意**：

在使用equals()方法进行比较时，两个对象如果数据不一致必为False。

In [120]:
df1 = pd.DataFrame({'c':['f',0,np.nan]})
df1

,c
0,f
1,0
2,NaN


In [121]:
df2 = pd.DataFrame({'c':[np.nan, 0, 'f']}, index=[2,1,0])
df2

,c
2,NaN
1,0
0,f


In [122]:
df1.equals(df2)

False

In [123]:
df1.equals(df2.sort_index()) #对df2的索引排序，然后再比较

True

### 不同类型的对象之间 逐元素比较

你可以直接对pandas对象和一个常量值进行逐元素比较：

In [124]:
pd.Series(['foo', 'bar', 'baz']) == 'foo'

0     True
1    False
2    False
dtype: bool

In [125]:
pd.Index(['foo', 'bar', 'baz']) == 'foo'

array([ True, False, False], dtype=bool)

不同类型的对象(比如pandas数据结构、numpy数组)之间进行逐元素的比较也是没有问题的，**前提是两个对象的shape要相同**。

In [128]:
pd.Series(['foo', 'bar', 'baz']) == pd.Index(['foo', 'bar', 'qux'])

0     True
1     True
2    False
dtype: bool

In [129]:
pd.Series(['foo', 'bar', 'baz']) == np.array(['foo', 'bar', 'qux'])

0     True
1     True
2    False
dtype: bool

In [130]:
pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo', 'bar']) #长度不相同

ValueError: Series lengths must match to compare

但要知道不同shape的numpy数组之间是可以直接比较的！因为广播！即使无法广播，也不会Error而是返回False。

In [133]:
np.array([1,2,3]) == np.array([2])

array([False,  True, False], dtype=bool)

In [134]:
np.array([1, 2, 3]) == np.array([1, 2])

c:\python27\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


False

###  combine_first()

看一下例子：

In [135]:
df1 = pd.DataFrame({'A' : [1., np.nan, 3., 5., np.nan],
                    'B' : [np.nan, 2., 3., np.nan, 6.]})
df1

,A,B
0,1.0,NaN
1,NaN,2.0
2,3.0,3.0
3,5.0,NaN
4,NaN,6.0


In [136]:
df2 = pd.DataFrame({'A' : [5., 2., 4., np.nan, 3., 7.],
                        'B' : [np.nan, np.nan, 3., 4., 6., 8.]})
df2

,A,B
0,5.0,NaN
1,2.0,NaN
2,4.0,3.0
3,NaN,4.0
4,3.0,6.0
5,7.0,8.0


In [137]:
df1.combine_first(df2)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


解释：

对于df1中NaN的元素，用df2中对应位置的元素替换！

### DataFrame.combine()

DataFrame.combine()方法接收一个DF对象和一个combiner方法。

In [140]:
combiner = lambda x,y: np.where(pd.isnull(x), y,x)

In [141]:
df1.combine(df2, combiner)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


## 统计相关 的方法

Series, DataFrame和Panel内置了许多计算统计相关指标的方法。这些方法大致分为两类：
* 返回低维结果，比如sum(),mean(),quantile()
* 返回同原对象同样大小的对象，比如cumsum(), cumprod()

总体来说，这些方法接收一个坐标轴参数:
* Series不需要坐标轴参数
* DataFrame 默认axis=0(index), axis=1(columns)
* Panel 默认axis=1(major), axis=0(items), axis=2(minor)

In [142]:
df

,one,three,two
a,-0.819889,NaN,1.000000
b,1.206005,-0.586788,0.630745
c,-0.620845,-0.080498,-0.227499
d,NaN,-1.495108,-1.590267


In [146]:
df.mean() #axis=0, 计算每一列的平均值

one     -0.078243
three   -0.720798
two     -0.046755
dtype: float64

In [147]:
df.mean(1) #计算每一行的平均值

a    0.090055
b    0.416654
c   -0.309614
d   -1.542688
dtype: float64

所有的这些方法都有skipna参数，含义是计算过程中是否剔除缺失值，skipna默认值为True。

In [148]:
df.sum(0, skipna=False)

one           NaN
three         NaN
two     -0.187021
dtype: float64

In [149]:
df.sum(axis=1, skipna=True)

a    0.180111
b    1.249962
c   -0.928841
d   -3.085375
dtype: float64

**这些函数可以参与算术和广播运算。**

比如：

In [152]:
ts_stand = (df-df.mean())/df.std()

In [153]:
ts_stand.std()

one      1.0
three    1.0
two      1.0
dtype: float64

In [154]:
xs_stand = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)

In [155]:
xs_stand.std(1)

a    1.0
b    1.0
c    1.0
d    1.0
dtype: float64

注意cumsum() cumprod()方法 保留NA值的位置。

In [156]:
df.cumsum()

,one,three,two
a,-0.819889,NaN,1.000000
b,0.386116,-0.586788,1.630745
c,-0.234729,-0.667286,1.403247
d,NaN,-2.162394,-0.187021
